In [73]:
import pandas as pd
import string

# 段落切分函數

In [74]:
def split_x(x,n=20):
    results = []
    if '+1)' in x:
        for i in range(1,n):
            text = x[x.find(f'+{i})'):x.find(f'+{i+1})')]
            if len(text) > 5:
                results.append(str(text))
    
    elif '1)' in x:
        for i in range(1,n):
            text = x[x.find(f'{i})'):x.find(f'{i+1})')]
            if len(text) > 5:
                results.append(str(text))
    
    elif '1.' in x:
        for i in range(1,n):
            text = x[x.find(f'{i}.'):x.find(f'{i+1}.')]
            if len(text) > 5:
                results.append(str(text))
    
    elif '+' in x:
        for text in x.split('+'):
            if len(text) > 5:
                results.append('+'+str(text))
    
    elif '(A)' in x:
        for i,j in zip(string.ascii_uppercase,string.ascii_uppercase[1:]+'A'):
            text = x[x.find(f'({i})'):x.find(f'({j})')]
            if len(text) > 5:
                results.append(str(text))
            
    return results

# 讀取押匯文件寶典

In [75]:
df = pd.read_excel('../data/20210831 押匯文件清單- 202007-202106.老師.xlsx')[['文件類別:','可能的說法']]
for i in df.columns:
    df[i] = df[i].apply(lambda x:x.replace('\n',''))
def string2list(s):
    return [ i.replace('\n','') for i in s.split('、')]
df['可能的說法'] = df['可能的說法'].apply(string2list)
def create_mapping(df):
    寶典 = {}
    for idx in df.index:
        d = {}
        for p in df.loc[idx,'可能的說法']:
            if p != '':
                d[p] = df.loc[idx,'文件類別:'].replace(' ','')
        寶典.update(d)
    return 寶典
寶典 = create_mapping(df)
寶典

{'BENEFICIARYS DRAFT': '匯票',
 'DRAFTS': '匯票',
 'BILL OF LADING': '提單',
 'TRANSPORT DOCUMENTS': '提單',
 'B/L': '提單',
 'MARINE/OCEAN BILL OF LADINGOCEANBILL OF LADING': '提單',
 'OCEAN BILL(S) OFLADING': '提單',
 'BILL(S) OF LADING': '提單',
 'AIR WAYBILLS': '提單',
 'COMMERCIAL INVOICE': '商業發票',
 'INVOICE': '商業發票',
 'INV.': '商業發票',
 'PACKING LIST': '裝箱單',
 'PACKING': '裝箱單',
 'P/L': '裝箱單',
 'INSURANCE POLICY/CERTIFICATE': '保險單據',
 'INSURANCE CERTIFICATE/POLICY': '保險單據',
 'I/PMARINE INSURANCE POLICY ': '保險單據',
 'MARINE CARGO POLICY': '保險單據',
 'CARGO POLICY (INSURANCE AGENCY)': '保險單據',
 'INSURANCE DOCUMENT': '保險單據',
 'INSURANCE POLICY OR CERTIFICATE': '保險單據',
 'Insurance Certificate': '保險單據',
 'CONSULAR DECLARATION': '領事簽證(C/I)',
 ' CONSULAR INVOICE': '領事簽證(C/I)',
 'CONSULAGE': '領事簽證(C/I)',
 'CONSULAR LEGALIZED INVOICE': '領事簽證(C/I)',
 'CERTIFICATE OF ORIGIN': '產地(商會)證明',
 'COOECFA DOCUMENT': '產地(商會)證明',
 'Certificate of Taiwan Origin': '產地(商會)證明',
 ' CERTIFICATE OF TAIWANESE  ORIGIN': '產地(商會)證明',
 

In [76]:
# 人工處理
#寶典['MARINE BILL OF LADINGOCEANBILL OF LADING'] = '提單'
#寶典['OCEAN BILL OF LADINGOCEANBILL OF LADING'] = '提單'
#寶典['INSURANCE POLICY'] = '保險單據'
#寶典['INSURANCE CERTIFICATE'] = '保險單據'

In [77]:
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)[['LCNO','46A','47A']].dropna(axis=0)
idx = df.sample(9).index
df.loc[idx]

,LCNO,46A,47A
20663,0AFOB2MA0442711,1. SIGNED COMMERCIAL INVOICES IN 3 COPIES INDI...,1. THE LANGUAGE OF ALL DOCUMENTS MUST BE IN EN...
20070,LC0610220000505,1.MANUALLY SIGNED COMMERCIAL INVOICE IN 2 ORIG...,1.ALL DOCUMENTS TO BE FORWARDED IN ONE COVER.\...
14965,S-331-2002153,+SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS.+COP...,+ALL DOCUMENTS MUST BE ISSUED IN ENGLISH LANGU...
58,0252FLC210002,+DRAFTS AT SIGHT+SIGNED COMMERCIAL INVOICE (S)...,+COMPLETE NAME AND ADDRESS OF APPLICANT:CASTOR...
18580,904IUBN202810002,+SIGNED COMMERCIAL INVOICE IN 1 ORIGINAL.\r\n+...,+DOCUMENTS MUST BE PRESENTED WITHIN 21 DAYS AF...
19439,0129IL2000645,1. SIGNED COMMERCIAL INVOICE ISSUED AND SIGNED...,1. ONE ADDITIONAL COPY/PHOTOCOPY OF ALL ORIGIN...
17699,49560NI00011220,1. DRAFTS IN DUPLICATE DRAWN ON US AT SIGHT\r\...,1.TRANSPORT DOCUMENT REQUIRED\r\nA.MUST NOT IN...
2373,CGBFSSDLC2100003,1.SIGNED COMMERCIAL INVOICE IN 3 ORIGINAL(S) A...,+ APPLICANT'S ADDRESS:\r\nSHUNDE FOSHAN\r\n1.E...
18822,0AFOB2MA0364703,1. SIGNED COMMERCIAL INVOICE IN TRIPLICATE.\r\...,1. APPLICANT CONFIRMS TO PAY FREIGHT SEPARATEL...


# 段落切分

In [78]:
x1 = []
x2 = []
for idx in df.index:
    lcno = df.loc[idx,'LCNO']
    x1 = x1 + [ lcno + ' ' + i for i in split_x(str(df.loc[idx,'46A']))]
    x2 = x2 + [ lcno + ' ' + i for i in split_x(str(df.loc[idx,'47A']))]
len(x1),len(x2)

(74569, 113515)

# 執行寶典搜索

In [79]:
from tqdm import tqdm_notebook as tqdm
def Collection_method(x,寶典):
    labels = {}
    for i in tqdm(x):
        products = []
        for p in 寶典:
            if p in i:
                products.append(p) 
            elif ('/' in p) & (len(p)>len('P/L')):
                if p.split('/')[0] in i:
                    products.append(p) 
                if p.split('/')[1] in i:
                    products.append(p) 
        try:
            labels[i] = max(products,key=len)
        except:
            labels[i] = 'not find'
    predict = pd.DataFrame(columns=['string_X_train','predict'])
    predict['string_X_train'] = labels.keys()
    predict['predict'] = labels.values()
    return predict

In [80]:
result1 = Collection_method(x1,list(寶典.keys()))
result2 = Collection_method(x2,list(寶典.keys()))
result1['from'] = '46A'
result2['from'] = '47A'
result = result1.append(result2)
result.head()

<ipython-input-79-0cb518c16de1>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(x):


,string_X_train,predict,from
0,08761FLC0003220 1. FULL SET OF(3 NEGOTIABLE CO...,SHIPPING COMPANYS/ SHIPPING AGENTS CERTIFICATE,46A
1,08761FLC0003220 2.BENEFICIARYS SIGNED COMMERCI...,INSURANCE CERTIFICATE/POLICY,46A
2,08761FLC0003220 3.PACKING LIST..,I/PMARINE INSURANCE POLICY,46A
3,08761FLC0003220 4.CERTIFICATE OF ORIGIN IN TWO...,INSURANCE POLICY/CERTIFICATE,46A
4,08761FLC0003220 5.ONE SET OF NON NEGOTIABLE DO...,INSURANCE POLICY/CERTIFICATE,46A


In [81]:
def k(x):
    return x.split(' ')[0]
def j(x):
    return ' '.join(x.split(' ')[1:])
result['LCNO'] = result['string_X_train'].apply(k)
result['string_X_train'] = result['string_X_train'].apply(j)
result

,string_X_train,predict,from,LCNO
0,1. FULL SET OF(3 NEGOTIABLE COPIES PLUS 3 NON-...,SHIPPING COMPANYS/ SHIPPING AGENTS CERTIFICATE,46A,08761FLC0003220
1,2.BENEFICIARYS SIGNED COMMERCIAL INVOICE(S) IN...,INSURANCE CERTIFICATE/POLICY,46A,08761FLC0003220
2,3.PACKING LIST..,I/PMARINE INSURANCE POLICY,46A,08761FLC0003220
3,4.CERTIFICATE OF ORIGIN IN TWO COPIES ISSUED B...,INSURANCE POLICY/CERTIFICATE,46A,08761FLC0003220
4,5.ONE SET OF NON NEGOTIABLE DOCUMENTS TO BE SE...,INSURANCE POLICY/CERTIFICATE,46A,08761FLC0003220
...,...,...,...,...
71308,+AN EXTRA COPY OF INVOICE AND TRANSPORT DOCUME...,I/PMARINE INSURANCE POLICY,47A,904IUSN203640001
71309,+THE AMOUNT OF EACH DRAWING MUST BE ENDORSED O...,I/PMARINE INSURANCE POLICY,47A,904IUSN203640001
71310,+A PROCESSING FEE OF USD70.00 OR ITS EQUIVALEN...,I/PMARINE INSURANCE POLICY,47A,904IUSN203640001
71311,+DOCUMENTARY PRESENTATION RECEIVED AT OUR COUN...,I/PMARINE INSURANCE POLICY,47A,904IUSN203640001


In [82]:
result['class'] = result['predict'].map(寶典)

In [83]:
result.sample(20)

,string_X_train,predict,from,LCNO,class
29588,+ SIGNED INVOICES IN 1 ORIGINAL.\r\n,I/PMARINE INSURANCE POLICY,46A,628010780034-M,保險單據
42196,+PACKING LIST IN TRIPLICATE\r\n,I/PMARINE INSURANCE POLICY,46A,M42BV2012NU00071,保險單據
1638,+ ALL BANKING CHARGES INCURRED AT BENEFICIARY'...,I/PMARINE INSURANCE POLICY,47A,1AY2/00033/7150,保險單據
11730,"7. CERTIFICATE OF ORIGIN, ORIGINAL NO-WOOD PAC...",INSURANCE POLICY/CERTIFICATE,46A,F1OBDK20012/2,保險單據
52752,+ FIELD 48 PERIOD OF PRESENTATION : WITHIN 21 ...,I/PMARINE INSURANCE POLICY,47A,DPCBOM001075,保險單據
61145,"5. ALL DOCUMENTS MUST HAVE, AT LEAST FOR ONE O...",I/PMARINE INSURANCE POLICY,47A,265C20LC007606,保險單據
25044,1. SIGNED COMMERCIAL INVOICE IN 3 COPIES CERTI...,I/PMARINE INSURANCE POLICY,46A,811-01-0127596,保險單據
19969,4.CERTIFICATE OF QUANTITY/WEIGHT IN 3 ORIGINAL...,INSURANCE POLICY/CERTIFICATE,46A,1ASOU200236BUST,保險單據
18278,"10)IF DOCUMENTS CONTAIN DISCREPANCY(S), DISCRE...",I/PMARINE INSURANCE POLICY,47A,1ASOU280008SUSV,保險單據
45983,6. DOCUMENTS PRESENTED TO ISSUING BANK UNDER T...,I/PMARINE INSURANCE POLICY,47A,26010370014277,保險單據


In [84]:
result[result['predict']!='not find'].shape

(115593, 5)

In [85]:
result[result['predict']=='not find'].shape

(1198, 5)

In [86]:
result.to_csv('../data/preprocess_for_SQUAD_押匯文件.csv')